# Benchmark an Endpoint

Welcome to this tutorial on benchmarking an endpoint deployed on a SambaNova dedicated node!

If you don't already have an endpoint deployed, please follow one of the workflows described in the [README](./README.md) to deploy an endpoint before proceeding with this tutorial.

Also, please install the benchmarking requirements in the Python kernel used in this Jupyter notebook

`! pip install -r ../../benchmarking/requirements.txt`

## 1.  Imports

In [2]:
import sys
sys.version

'3.11.11 (main, Dec 11 2024, 10:28:39) [Clang 14.0.6 ]'

In [3]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))
import json
import os
import pprint
import getpass
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
benchmarking_dir = "../../benchmarking/"
sys.path.append(benchmarking_dir + "src")
sys.path.append(benchmarking_dir + "benchmarking_scripts")
sys.path.append(benchmarking_dir + "../")
from synthetic_performance_eval_script import *
from benchmarking.utils import read_perf_eval_json_files

In [ ]:
class CustomPromptsPerformanceEvaluator(BasePerformanceEvaluator):
    def __init__(
        self,
        num_concurrent_requests: int,
        input_file_path: str,
        save_response_texts: bool = False,
        prompt_ids: List[str] = [],
        *args: Any,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.num_concurrent_requests = num_concurrent_requests
        self.file_name = os.path.basename(input_file_path)
        self.dataset, self.prompt_key, self.img_path_key, self.prompt_ids = self.read_dataset(input_file_path)         
        self.save_response_texts = save_response_texts
        

    @staticmethod
    def read_dataset(input_file_path: str) -> List[Dict[str, Any]]:
        """Utility function for reading in the `.jsonl` file provided by the user for custom dataset evaluation.

        Args:
            input_file_path (str): The absolute file path of the input file provided by the user

        Returns:
            List[Dict]: A list of json objects (python dictionaries) containing the individual prompts the user wants
            to evaluate on
        """
        with open(input_file_path, 'r') as file:
            data = json.load(file)

        # check if dict keys contain 'prompt'
        prompt_ids = list(data.keys())
        img_path_key = None
        for prompt_id in prompt_ids:
            d = data[prompt_id]
            if 'prompt' not in d:
                raise ValueError(
                    'All entries in input file must contain the "prompt" key \
                    and its respective text value'
                )

            # check if dict keys contain 'img_path' if there are two keys
            if len(d) == 2:
                if 'image_path' not in data_entry:
                    raise ValueError(
                        'All entries in input file that contain two keys must contain "image_path" \
                        as a key in addition to the "prompt" key and its respective text encoding value'
                    )
                if d['image_path'].startswith('http'):
                    raise ValueError('Urls are not supported for image_path. Please provide text encodings.')                
                img_path_key = 'image_path'

            if len(d) > 2:
                raise ValueError('All entries in input file must contain at most two keys: "prompt" and "image_path".')

        return data, "prompt", img_path_key, prompt_ids     

    def create_output_filename(self, prompt_id: str, num_output_tokens: int) -> str:
        """Utility for creating a unique filename for a synthetic benchmarking experiment given user specified params.

        Args:
            prompt_id (str): The user-specified identifier for the prompt in the input file.
            num_output_tokens (int): The number of output tokens to be received.

        Returns:
            str: Filename for the synthetic benchmark run.
        """
        generation_mode = ''
        if self.is_stream_mode:
            generation_mode = 'stream'

        multimodal_suffix = ''
        if self.multimodal_image_size != 'na':
            multimodal_suffix = f'_multimodal_{self.multimodal_image_size}'

        output_file_name = (
            f'custom_{self.user_metadata["model_idx"]}_{self.model_name}{multimodal_suffix}_{prompt_id}'
            f'_{num_output_tokens}_{self.num_concurrent_requests}_{generation_mode}_{self.run_uuid}'
        )

        return self.sanitize_file_prefix(output_file_name)

    def save_results(
        self,
        filename: str,
        summary: Dict[str, Any],
        individual_responses: (
            List[LLMResponse]
            | List[Tuple[Dict[str, Any], str, RequestConfig]]
            | Tuple[Dict[str, object], List[LLMResponse]]
        ),
    ) -> None:
        """Save the performance evaluation results to a file, and completion texts if save_response_text condition is
        setup as True

        Args:
            filename (str): The base name of the file to save the results to.
            summary (Dict[str, Any]): A dictionary containing the summary of the performance evaluation.
            individual_responses (List[LLMResponse]): A list of individual responses from the performance evaluation.

        Raises:
            e: if an error happens when creating the output file related to prompts and completions, an error will be
            raised
        """

        super().save_results(filename, summary, individual_responses)

        # If specified, save the llm responses to output file
        if self.save_response_texts:
            # Create response texts file name
            response_texts_file_name = f'{filename}_response_texts'
            results_dir = Path(self.results_dir)

            # Save response texts
            try:
                self.response_texts_file_path = f'{results_dir}/{response_texts_file_name}.jsonl'
                with open(self.response_texts_file_path, 'w') as f:
                    for response in individual_responses:
                        if isinstance(response, LLMResponse):
                            output_json = {
                                'prompt': response.request_config.prompt_tuple[0],
                                'completion': str(response.response_text),
                            }
                            f.write(json.dumps(output_json))
                            f.write('\n')
            except Exception as e:
                logger.error('ERROR SAVING LLM OUTPUTS')
                raise e

    def run_benchmark(
        self, sampling_params: Dict[str, Any] = {}, *args: Any, **kwargs: Any
    ) -> Tuple[Dict[str, Any], List[LLMResponse]]:
        """Run a benchmark test for the specified LLM using synthetically generated data.

        Args:
            prompt_id (str): The user-specified identifier for the prompt in the input file.
            num_output_tokens (int): The number of output tokens to be received.
            num_requests (int): The number of requests to be made.
            sampling_params (str): The sampling parameters in JSON format.

        Raises:
            ValueError: If the number of input tokens is less than 40.

        Returns:
            summary (dict): structure with performance metrics and stats for the run
            individual_responses (tuple): list of performance metrics per request
        """
        self.run_uuid = uuid.uuid4()
        prompt_id = kwargs.get('prompt_id', "0")
        num_output_tokens = kwargs.get('num_output_tokens', 10)
        num_requests = kwargs.get('num_requests', 1)

        self.cli_progress_bar = tqdm(total=num_requests, desc='Running Requests')
        self.ui_progress_bar = kwargs.get('progress_bar', None)

        # Calculate performance metrics individually and summary
        summary, individual_responses = self.get_token_throughput_latencies(
            prompt_id=prompt_id,
            num_output_tokens=num_output_tokens,
            num_requests=num_requests,
            sampling_params=sampling_params,
        )

        if self.results_dir:
            filename = self.create_output_filename(prompt_id, num_output_tokens)
            self.save_results(filename, summary, individual_responses)

        return summary, individual_responses

    def add_metric_after_key(
        self,
        metrics_dict: Dict[str, Any],
        new_key: str,
        new_value: float,
        after_key: str,
    ) -> Dict[str, Any]:
        """Adds a new metric (dict key and value) to a dict after an specific key

        Args:
            metrics_dict (dict): dictionary to add new metric
            new_key (str): new key
            new_value (float): new value for key
            after_key (str): key for reference to add new key after

        Returns:
            dict: dictionary with new key and value added
        """

        # Create a new dictionary
        new_metrics_dict = {}

        for key, value in metrics_dict.items():
            # Copy the key-value pair to the new dictionary
            new_metrics_dict[key] = value

            # Check if this is the key after which to insert the new key-value pair
            if key == after_key:
                new_metrics_dict[new_key] = new_value

        return new_metrics_dict

    def calculate_switching_time(self, llm_responses: list[LLMResponse]) -> list[LLMResponse]:
        """Logic to calculate switching time. Based on the first request TTFT,
        if this value is significantly larger (more than 3 standard deviations) than the average TTFT
        of the rest requests, then switching time will be the difference between first TTFT
        and average of the coming TTFTs.

        Args:
            llm_responses (list[LLMResponse]): list of LLMResponse objects

        Returns:
            list[LLMResponse]: list of LLMResponse objects including switching time
        """
        # collect necessary information for switching time calculation
        responses_ttfts = []

        for llm_response in llm_responses:
            if pd.isnull(llm_response.metrics['error_code']):
                request_idx = llm_response.request_config.request_idx
                start_time = llm_response.metrics['start_time']
                server_ttft_s = llm_response.metrics['server_ttft_s']
                responses_ttfts.append(
                    {
                        'request_idx': request_idx,
                        'start_time': start_time,
                        'server_ttft_s': server_ttft_s,
                    }
                )

        df_valid_responses = pd.DataFrame(responses_ttfts)

        # transforming str to date time for sorting
        df_valid_responses['start_time'] = pd.to_datetime(df_valid_responses['start_time'])
        df_valid_responses = df_valid_responses.sort_values(by=['start_time'])

        # initialize a column for the switching time
        df_valid_responses['server_switching_time'] = None

        # check server ttft in case metric is not coming in response
        if df_valid_responses['server_ttft_s'].notna().all():
            # calculate switching time
            first_ttft = df_valid_responses['server_ttft_s'].iloc[0]
            mean_ttft = df_valid_responses['server_ttft_s'].iloc[1:].mean()
            std_ttft = df_valid_responses['server_ttft_s'].iloc[1:].std()
            std_ttft = 1e-16 if np.isnan(std_ttft) else std_ttft

            switching_time = first_ttft - mean_ttft
            outlier_switching_time = None

            if switching_time > (mean_ttft + 3 * std_ttft):
                outlier_switching_time = switching_time
                df_valid_responses['server_switching_time'].iloc[0] = outlier_switching_time

        # assign switching time back to request object
        for llm_response in llm_responses:
            metrics = llm_response.metrics

            if llm_response.request_config.request_idx == df_valid_responses.head(1)['request_idx'].values[0]:
                server_switching_time = df_valid_responses.head(1)['server_switching_time'].values[0]
            else:
                server_switching_time = None

            llm_response.metrics = self.add_metric_after_key(
                metrics,
                new_key='server_switching_time',
                new_value=server_switching_time,
                after_key=common_metrics.TTFT_SERVER,
            )

        return llm_responses

    def get_token_throughput_latencies(
        self,
        prompt_id: str,
        num_output_tokens: int,
        num_requests: int,
        sampling_params: Dict[str, Any],
    ) -> Tuple[dict[str, Any], List[LLMResponse]]:
        """This function runs a token benchmark for the given model and API,
        measuring the throughput and latencies for the specified number of input and output tokens,
        and the specified number of requests.

        Args:
            prompt_id (str): The user-specified identifier for the prompt in the input file.
            num_output_tokens (int): The user specified number of output tokens.
            num_requests (int): The user specified number of requests to run.
            sampling_params (dict): User specified sampling parameters for generation.

        Returns:
            metadata (dict): A dictionary containing the results of the benchmark,
                            including the model name, number of concurrent requests,
                            results, number of input tokens, number of output tokens,
                            and additional sampling parameters.
            completed_requests (list): A list of completed requests.

        Raises:
            Exception: If an unexpected error occurs during the execution of requests.
        """
        # random.seed(11111)

        # Build the request config objects that are to be sent to the LLM API endpoint
        request_configs = self.build_request_configs(num_requests, prompt_id, num_output_tokens, sampling_params)

        # Get the request counts in order to place them into threads to be executed in batches
        total_request_count = len(request_configs)
        request_config_batches: List[List[RequestConfig]] = []

        if self.num_concurrent_requests:
            requests_per_thread = (total_request_count) // self.num_concurrent_requests
            remainder = (total_request_count) % self.num_concurrent_requests

            idx = 0
            # Create batches of requests for each concurrent request
            for concurrent_requests in range(self.num_concurrent_requests):
                num_requests_for_thread = requests_per_thread + (1 if concurrent_requests < remainder else 0)
                request_config_batch = request_configs[idx : idx + num_requests_for_thread].copy()
                idx += num_requests_for_thread
                request_config_batches.append(request_config_batch)

        # Execute requests concurrently
        llm_responses: List[LLMResponse] = []
        progress: List[Any] = []

        start_time = time.monotonic()
        # Use ThreadPoolExecutor to handle threads
        with ThreadPoolExecutor(max_workers=self.num_concurrent_requests) as executor:
            # Store futures for the tasks
            futures = []

            for request_config_batch in request_config_batches:
                if self.stop_event.is_set():
                    logger.info('Stopping task submission due to stop signal.')
                    break
                # Submit the task to the executor
                future = executor.submit(
                    self.send_requests,
                    request_config_batch,
                    llm_responses,
                    progress,
                    start_time,
                    num_requests,
                )
                futures.append(future)
                for t in executor._threads:
                    add_script_run_ctx(t)

            # Wait for all tasks to complete
            for future in as_completed(futures):
                try:
                    # Retrieve result if needed
                    future.result()
                except Exception as e:
                    logger.error(f'Error occurred in a thread: {e}')

        if self.stop_event.is_set():
            logger.info('Benchmarking process terminated early due to stop signal.')
            return {}, []

        # Error handling
        error_codes = [llm_response.metrics['error_code'] for llm_response in llm_responses]

        if not any([pd.isnull(error_code) for error_code in error_codes]):
            unique_error_codes = list(
                set(
                    [
                        llm_response.metrics['error_code']
                        for llm_response in llm_responses
                        if not pd.isnull(llm_response.metrics['error_code'])
                    ]
                )
            )
            unique_error_msgs = list(
                set(
                    [
                        llm_response.metrics['error_msg']
                        for llm_response in llm_responses
                        if not pd.isnull(llm_response.metrics['error_code'])
                    ]
                )
            )
            nl = '\n'
            raise Exception(
                f"""Unexpected error happened when executing requests:\
                {nl}{f'{nl}'.join([f'- {error_code}' for error_code in unique_error_codes])}\
                {nl}Additional messages:{nl}{f'{nl}'.join([f'- {error_msg}' for error_msg in unique_error_msgs])}"""
            )

        # Capture end time and notify user
        end_time = time.monotonic()
        logger.info('Tasks Executed!')
        logger.info(f'Benchmarking results obtained for model {self.model_name} queried with the {self.llm_api} API.')

        # Calculate switching time
        llm_responses = self.calculate_switching_time(llm_responses)

        # Build a metrics summary for the results of the benchmarking run
        results = self.build_metrics_summary(
            metrics=[response.metrics for response in llm_responses],
            start_time=start_time,
            end_time=end_time,
        )

        # Construct metadata payload to be returned
        metadata = {
            'model': self.model_name,
            'num_concurrent_requests': self.num_concurrent_requests,
            'results': results,
            'prompt_id': prompt_id,
            'num_output_tokens': num_output_tokens,
            'additional_sampling_params': sampling_params,
        }

        return metadata, llm_responses

    def select_raw_prompts(self, raw_prompts: List[Dict[str, Any]], num_requests: int) -> List[Dict[str, Any]]:
        """Selects prompts randomly

        Args:
            num_requests (int): Number of requests to be generated

        Returns:
            List[Dict[str,Any]]: List of randomly selected prompts
        """

        random_selected_prompts = random.choices(raw_prompts, k=num_requests)
        assert len(random_selected_prompts) == num_requests, 'Number of selected prompts \
            does not match the requested count'
        return random_selected_prompts

    def build_request_configs(
        self,
        num_requests: int,
        prompt_id: str,
        output_token_count: int,
        sampling_params: Dict[str, Any],
    ) -> List[RequestConfig]:
        """Builds a list of request configuration objects used to send requests to the LLM. It iterates through the
        specified number of requests, builds an input prompt for each request, updates the sampling parameters with
        the maximum number of tokens to generate, and then creates the request configuration object. The request
        configurations are then returned as a list.

        Args:
            num_requests (int): The number of request configurations to build.
            prompt_id (str): The user-specified identifier for the prompt in the input file.
            output_token_count (int): The number of output tokens each request should return.
            sampling_params (dict): A dictionary of sampling parameters for the LLM.

        Returns:
            List[RequestConfig]: A list of request configurations, each containing the model name, prompt, sampling
            parameters, LLM API, generation mode, and number of concurrent requests.
        """
        # Empty list to be filled with valid request configs and then returned
        request_configs = []
        # Instantiate image variable
        image = None

        data_point = self.dataset[prompt_id]

        raw_prompt = {'name': 'custom_prompt', 'template': data_point[self.prompt_key]}

        # Apply prompt templating to get final prompt to send to LLM API along with tokenized prompt length
        prompt_tuple = (raw_prompt, self.get_token_length(raw_prompt["template"]))

        # Image to be sent in LLM request if exists
        image = None
        if self.img_path_key:
            image = self.get_image(data_point[self.img_path_key])

        # Iterate through data points and build a request config for each
        for request_idx in range(num_requests):
            # Add generic max tokens parameter to `sampling_params` dictionary
            updated_sampling_params = {
                'max_tokens_to_generate': output_token_count,
            }
            updated_sampling_params.update(sampling_params)

            # Create request config object
            request_config = RequestConfig(
                request_idx=request_idx,
                model=self.model_name,
                prompt_tuple=prompt_tuple,
                image=image,
                sampling_params=updated_sampling_params,
                llm_api=self.llm_api,
                api_variables=self.api_variables,
                is_stream_mode=self.is_stream_mode,
                num_concurrent_requests=self.num_concurrent_requests,
            )

            request_configs.append(request_config)

        return request_configs
    
    def build_prompt(self, *args: Any, **kwargs: Any) -> Tuple[Dict[str, Any], int]:
        pass


## 2. Get endpoint info
To benchmark the endpoint, we will need to obtain some of its information. Note that this information can be obtained from your SambaNova representative.

#### Enter the endpoint url
Run the cell below and then enter the endpoint URL. This should be in the format of `https://my.env/v1/<endpoint_id>/chat/completions` 

In [4]:
endpoint_url = input().strip()
if endpoint_url[-1] == "/":
    endpoint_url = endpoint_url[:-1]
os.environ["SAMBASTUDIO_URL"] = endpoint_url
endpoint_id = endpoint_url.split('/')[-3]
print("Benchmarking Endpoint:", endpoint_id)

Benchmarking Endpoint: 631938c4-1336-40fd-aa08-9683b4dda499


#### Enter the endpoint API key

In [5]:
endpoint_key = getpass.getpass().strip()
if len(endpoint_key) > 0:
    os.environ["SAMBASTUDIO_API_KEY"] = endpoint_key
else:
    print("Please enter a valid key")

## 3. Automatically retrieve model list from endpoint (Optional)
Run this section only if you don't the have list of models on the endpoint. Note that running this section requires you to first follow the set up instructions given in the [README](./README.md)
#### Set up environment connector
The connector connects to the remote dedicated environment using the variables defined below

In [ ]:
env_url = '/'.join(endpoint_url.split('/')[:3])

'https://sjc3-e9.sambanova.net'

In [7]:
print("Enter the env access key")
env_key = getpass.getpass().strip()
if len(env_key) > 0:
    os.environ["SAMBASTUDIO_ACCESS_KEY"] = env_key
else:
    print("Please enter a valid key")

Enter the env access key


In [8]:
env_tenant = "default"

In [9]:
from snsdk import SnSdk
sn_env = SnSdk(host_url=env_url, 
                   access_key=env_key, 
                   tenant_id=env_tenant)

#### Get model names in the endpoint

In [10]:
endpoint_info = sn_env.endpoint_info_by_id(endpoint_id)
endpoint_model_id = endpoint_info['targets'][0]["model"]
model_info = sn_env.model_info(endpoint_model_id, job_type="deploy")
model_constituents = [m["name"] for m in model_info["dependencies"]]
sorted(model_constituents)

['Meta-Llama-3.3-70B-Instruct',
 'Qwen-2.5-72B-SD-Qwen-2.5-0.5B',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r']

#### Get target model names in the endpoint
Target model names generally differ from model names when the model is a speculative decoding pair

In [11]:
target_models = []
for constituent_name in model_constituents:    
    model_name = constituent_name

    # Check for speculative decoding
    constituent_info = sn_env.model_info(constituent_name, job_type="deploy")
    if 'target_model' in constituent_info['config']:
        target_name = constituent_info['config']['target_model']        
        if len(target_name) > 0:
            model_name = target_name
    target_models.append(model_name)
sorted(target_models)

['Meta-Llama-3.3-70B-Instruct',
 'Qwen2.5-72B-Instruct',
 'Salesforce--Llama-xLAM-2-70b-fc-r',
 'Salesforce--Llama-xLAM-2-8b-fc-r']

## 4. Set up Model Configs for Benchmarking
Note that this section only currently supports a fraction of what the Benchmarking Kit is capable of. You may repurpose this section if you would like to benchmark images or run questions per second (qps).

#### Name the benchmarking run
Give the run a unique name so that the configs and results can be saved with that name. Please note that the name should be compatible with file system path naming rules.

In [5]:
run_name = "llama33_70b_sd_1b_short_studio_sfdata_20250602_1"

#### Specify target models
If not automatically set in Step 3, set Target models as a python list

In [6]:
target_models = ["Meta-Llama-3.3-70B-Instruct"] #target_models 

#### Specify custom input path

In [7]:
custom_input_path = os.path.expanduser("~/Downloads/custom_prompt_data.json")

#### Specify configs for benchmarking

In [8]:
output_path = f"{benchmarking_dir}data/benchmarking_tracking_tests/"

config = {
    'sampling_params': {},
    'max_tokens_to_generate': 1000, # treated as a special sampling param and not to be included in sampling_params  
    'output_files_dir': os.path.join(f'{output_path}logs/output_files/', run_name), # each run saved here
    'consolidated_results_dir': os.path.join(f'{output_path}consolidated_results', run_name), # consolidated xlsx saved here
    'timeout': 3600,
    'num_requests': 64,
    'concurrent_requests': [1, 2, 4, 8, 16, 32],
    'custom_input_path': custom_input_path,
    'save_response_texts': False,
    'llm_api': 'sambastudio'
}

## 5. Run Benchmarking
We will run benchmarking with the configs now

#### Benchmarking produces json files with detailed results for each configuration

In [12]:
for target_model in target_models:
    for num_concurrent_requests in config['concurrent_requests']:        
        custom_evaluator = CustomPromptsPerformanceEvaluator(
            model_name=target_model,        
            results_dir=config['output_files_dir'],
            num_concurrent_requests=num_concurrent_requests,
            timeout=config["timeout"],
            user_metadata={"model_idx": 0},
            input_file_path=config['custom_input_path'],
            save_response_texts=config['save_response_texts'],
            llm_api=config['llm_api'],
        )

        for prompt_id in custom_evaluator.prompt_ids:
            # Run performance evaluation
            custom_evaluator.run_benchmark(
                prompt_id=prompt_id,
                num_output_tokens=config["max_tokens_to_generate"],
                num_requests=config["num_requests"],
                sampling_params=config['sampling_params'])

Running Requests: 100%|██████████| 64/64 [01:55<00:00,  1.96s/it]

2025-06-02 15:13:06,741 [INFO] Tasks Executed!
2025-06-02 15:13:06,742 [INFO] Benchmarking results obtained for model Meta-Llama-3.3-70B-Instruct queried with the sambastudio API.
2025-06-02 15:13:06,772 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-06-02 15:13:06,777 [INFO]     p5 = 0.1849
2025-06-02 15:13:06,777 [INFO]     p25 = 0.2005
2025-06-02 15:13:06,777 [INFO]     p50 = 0.2808
2025-06-02 15:13:06,778 [INFO]     p75 = 0.3374
2025-06-02 15:13:06,778 [INFO]     p90 = 0.4858
2025-06-02 15:13:06,779 [INFO]     p95 = 0.549
2025-06-02 15:13:06,779 [INFO]     p99 = 0.7479
2025-06-02 15:13:06,780 [INFO]     mean = 0.3037
2025-06-02 15:13:06,781 [INFO]     min = 0.1686
2025-06-02 15:13:06,781 [INFO]     max = 0.7583
2025-06-02 15:13:06,781 [INFO]     stddev = 0.1298
2025-06-02 15:13:06,782 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-06-02 15:13:06,783 [INFO]     p5 = 1.6367
2025-06-02 15:13:06,783 [INFO]     p25 = 1.6861
202

Running Requests: 100%|██████████| 64/64 [01:55<00:00,  1.81s/it]


2025-06-02 15:16:44,813 [INFO] Tasks Executed!
2025-06-02 15:16:44,813 [INFO] Benchmarking results obtained for model Meta-Llama-3.3-70B-Instruct queried with the sambastudio API.
2025-06-02 15:16:44,823 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-06-02 15:16:44,824 [INFO]     p5 = 0.1849
2025-06-02 15:16:44,825 [INFO]     p25 = 0.2396
2025-06-02 15:16:44,825 [INFO]     p50 = 0.3272
2025-06-02 15:16:44,826 [INFO]     p75 = 0.4185
2025-06-02 15:16:44,827 [INFO]     p90 = 0.528
2025-06-02 15:16:44,827 [INFO]     p95 = 0.6694
2025-06-02 15:16:44,828 [INFO]     p99 = 0.9654
2025-06-02 15:16:44,829 [INFO]     mean = 0.355
2025-06-02 15:16:44,829 [INFO]     min = 0.1713
2025-06-02 15:16:44,830 [INFO]     max = 0.9903
2025-06-02 15:16:44,830 [INFO]     stddev = 0.1709
2025-06-02 15:16:44,830 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-06-02 15:16:44,832 [INFO]     p5 = 3.2136
2025-06-02 15:16:44,832 [INFO]     p25 = 3.2727
2025

Running Requests: 100%|██████████| 64/64 [04:06<00:00,  3.89s/it]

...

2025-06-02 16:17:29,795 [INFO] Tasks Executed!
2025-06-02 16:17:29,795 [INFO] Benchmarking results obtained for model Meta-Llama-3.3-70B-Instruct queried with the sambastudio API.
2025-06-02 16:17:29,805 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-06-02 16:17:29,806 [INFO]     p5 = 6.3485
2025-06-02 16:17:29,807 [INFO]     p25 = 6.3609
2025-06-02 16:17:29,807 [INFO]     p50 = 11.6371
2025-06-02 16:17:29,807 [INFO]     p75 = 11.74
2025-06-02 16:17:29,808 [INFO]     p90 = 13.201
2025-06-02 16:17:29,808 [INFO]     p95 = 13.2018
2025-06-02 16:17:29,808 [INFO]     p99 = 13.2085
2025-06-02 16:17:29,809 [INFO]     mean = 10.5293
2025-06-02 16:17:29,809 [INFO]     min = 0.5215
2025-06-02 16:17:29,809 [INFO]     max = 13.214
2025-06-02 16:17:29,810 [INFO]     stddev = 2.8967
2025-06-02 16:17:29,810 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-06-02 16:17:29,811 [INFO]     p5 = 8.4271
2025-06-02 16:17:29,811 [INFO]     p25 = 8.4367

Running Requests: 100%|██████████| 64/64 [00:29<00:00,  2.20it/s]


2025-06-02 16:18:06,450 [INFO] Tasks Executed!
2025-06-02 16:18:06,451 [INFO] Benchmarking results obtained for model Meta-Llama-3.3-70B-Instruct queried with the sambastudio API.
2025-06-02 16:18:06,459 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-06-02 16:18:06,461 [INFO]     p5 = 7.3168
2025-06-02 16:18:06,461 [INFO]     p25 = 7.3265
2025-06-02 16:18:06,461 [INFO]     p50 = 13.2671
2025-06-02 16:18:06,462 [INFO]     p75 = 13.292
2025-06-02 16:18:06,462 [INFO]     p90 = 15.7636
2025-06-02 16:18:06,462 [INFO]     p95 = 15.7667
2025-06-02 16:18:06,462 [INFO]     p99 = 15.7699
2025-06-02 16:18:06,463 [INFO]     mean = 12.1686
2025-06-02 16:18:06,463 [INFO]     min = 0.6063
2025-06-02 16:18:06,463 [INFO]     max = 15.7746
2025-06-02 16:18:06,463 [INFO]     stddev = 3.4325
2025-06-02 16:18:06,464 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-06-02 16:18:06,465 [INFO]     p5 = 10.989
2025-06-02 16:18:06,465 [INFO]     p25 = 11.

Running Requests: 100%|██████████| 64/64 [01:17<00:00,  3.43s/it]

2025-06-02 16:19:23,599 [INFO] Tasks Executed!
2025-06-02 16:19:23,599 [INFO] Benchmarking results obtained for model Meta-Llama-3.3-70B-Instruct queried with the sambastudio API.
2025-06-02 16:19:23,608 [INFO] Building Client Metrics Summary for metric: client_ttft_s
2025-06-02 16:19:23,610 [INFO]     p5 = 17.8018
2025-06-02 16:19:23,610 [INFO]     p25 = 17.8217
2025-06-02 16:19:23,610 [INFO]     p50 = 33.1324
2025-06-02 16:19:23,611 [INFO]     p75 = 33.2169
2025-06-02 16:19:23,611 [INFO]     p90 = 36.2289
2025-06-02 16:19:23,611 [INFO]     p95 = 36.2388
2025-06-02 16:19:23,612 [INFO]     p99 = 36.2436
2025-06-02 16:19:23,612 [INFO]     mean = 29.5425
2025-06-02 16:19:23,612 [INFO]     min = 1.1484
2025-06-02 16:19:23,613 [INFO]     max = 36.2462
2025-06-02 16:19:23,613 [INFO]     stddev = 8.0664
2025-06-02 16:19:23,613 [INFO] Building Client Metrics Summary for metric: client_end_to_end_latency_s
2025-06-02 16:19:23,614 [INFO]     p5 = 21.5764
2025-06-02 16:19:23,614 [INFO]     p25 =

#### Consolidate all results into pandas dataframe

In [11]:
output_files_dir = config['output_files_dir']
df_summary = read_perf_eval_json_files(output_files_dir, type='summary')
sorted(df_summary.columns)

['acceptance_rate_max',
 'acceptance_rate_mean',
 'acceptance_rate_min',
 'acceptance_rate_p25',
 'acceptance_rate_p5',
 'acceptance_rate_p50',
 'acceptance_rate_p75',
 'acceptance_rate_p90',
 'acceptance_rate_p95',
 'acceptance_rate_p99',
 'acceptance_rate_stddev',
 'client_end_to_end_latency_s_max',
 'client_end_to_end_latency_s_mean',
 'client_end_to_end_latency_s_min',
 'client_end_to_end_latency_s_p25',
 'client_end_to_end_latency_s_p5',
 'client_end_to_end_latency_s_p50',
 'client_end_to_end_latency_s_p75',
 'client_end_to_end_latency_s_p90',
 'client_end_to_end_latency_s_p95',
 'client_end_to_end_latency_s_p99',
 'client_end_to_end_latency_s_stddev',
 'client_mean_output_token_per_s',
 'client_output_token_per_s_max',
 'client_output_token_per_s_mean',
 'client_output_token_per_s_min',
 'client_output_token_per_s_p25',
 'client_output_token_per_s_p5',
 'client_output_token_per_s_p50',
 'client_output_token_per_s_p75',
 'client_output_token_per_s_p90',
 'client_output_token_per_s

In [12]:
missing_columns = []

if 'num_concurrent_requests' not in df_summary.columns:
    missing_columns.append('num_concurrent_requests')

if 'multimodal_img_size' not in df_summary.columns or df_summary['multimodal_img_size'].isnull().all():
    missing_columns.append('multimodal_img_size')

# Set fields to report
selected_columns = [
    'name',
    'model',    
    'number_input_tokens_min',
    'number_input_tokens_max',
    'number_output_tokens_min',
    'number_output_tokens_max',
    'num_concurrent_requests',
    'multimodal_img_size',
    'server_ttft_s_min',
    'server_ttft_s_p50',
    'server_ttft_s_max',
    'server_end_to_end_latency_s_min',
    'server_end_to_end_latency_s_p50',
    'server_end_to_end_latency_s_max',
    'server_output_token_per_s_min',
    'server_output_token_per_s_p50',
    'server_output_token_per_s_max',
    'server_output_token_per_s_mean',                
    'acceptance_rate_min',
    'acceptance_rate_p50',
    'acceptance_rate_max',
    'server_number_input_tokens_p50',
    'server_number_output_tokens_p50',
    'client_ttft_s_min',
    'client_ttft_s_p50',
    'client_ttft_s_max',
    'client_end_to_end_latency_s_min',
    'client_end_to_end_latency_s_p50',
    'client_end_to_end_latency_s_max',
    'client_output_token_per_s_min',
    'client_output_token_per_s_p50',
    'client_output_token_per_s_max',
    'client_output_token_per_s_mean',
    'client_mean_output_token_per_s',
    'num_requests_started',
    'num_completed_requests',
    'num_completed_requests_per_min',
    'number_errors',
    'error_code_frequency',
]

selected_columns = [c for c in selected_columns if c not in missing_columns]
# Set fields to report
df_summary = df_summary[selected_columns]
df_summary['model'] = df_summary['model'].str.replace('.', '-')
df_summary['requests_grouping'] = pd.Series(None, index=df_summary.index, dtype=object)
df_summary['requests_batching'] = pd.Series(None, index=df_summary.index, dtype=object)

# Add UUID to summary and set as index
#df_summary['uuid'] = df_summary.apply(lambda x: find_uuid(x['name']), axis=1)
df_summary = df_summary.set_index('name')

# Read individual responses
df = read_perf_eval_json_files(output_files_dir, type='individual_responses')

# Process individual files and add requests batching approximation
for filename in os.listdir(output_files_dir):
    if 'individual_responses' in filename:
        df_file = df[df['filename'] == filename].copy()
        df_file = df_file[df_file['error_code'].isnull()]

        requests_grouping, requests_batching = get_grouping_and_batching_info(df_file)

        key = filename[:filename.index("_individual_responses.json")] + "_summary"
        total_input_tokens = df_file["number_input_tokens"].sum()
        total_output_tokens = df_file["number_output_tokens"].sum()

        if key in df_summary.index:
            df_summary.at[key, 'requests_grouping'] = requests_grouping
            df_summary.at[key, 'requests_batching'] = requests_batching
            df_summary.at[key, 'total_input_tokens'] = total_input_tokens
            df_summary.at[key, 'total_output_tokens'] = total_output_tokens
        else:
            raise KeyError(f'Key {key} not found in dictionary. File: {file}')
df_summary['representative_batch_size'] = df_summary['requests_batching'].apply(
    lambda x: find_median_in_batches(x)
)

# Sort and save the summary DataFrame
consolidated_results_dir = os.path.expanduser(config['consolidated_results_dir'])
if not os.path.exists(consolidated_results_dir):
    os.makedirs(consolidated_results_dir)

df_summary["number_input_tokens_min"] = df_summary["number_input_tokens_min"].astype(int)
df_summary["number_output_tokens_min"] = df_summary["number_output_tokens_min"].astype(int)

sort_columns = ['model', 'num_concurrent_requests', 'number_input_tokens_min', 'number_output_tokens_min']
if 'num_concurrent_requests' in df_summary.columns:
    sort_columns.append('num_concurrent_requests')
df_summary.sort_values(by=sort_columns, inplace=True)

df_summary["server_rated_throughput"] = df_summary["server_output_token_per_s_mean"] * df_summary["representative_batch_size"]
df_summary["total_latency"] =  df_summary["num_completed_requests"]*60/df_summary["num_completed_requests_per_min"]
df_summary["client_output_throughput"] = df_summary["total_output_tokens"]/df_summary["total_latency"]
df_summary["client_total_throughput"] = (df_summary["total_input_tokens"]+df_summary["total_output_tokens"])/df_summary["total_latency"]

df_summary.to_excel(os.path.join(consolidated_results_dir, f'{run_name}.xlsx'))

## 6. Compare with alternative benchmarking results

In [13]:
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

In [15]:
current_name = "Llama 3.3 70B Standalone"
alt_name = "XLAM Standalone"

#### Read results from current run

In [16]:
current_file_path = os.path.join(config["consolidated_results_dir"], f'{run_name}.xlsx')
df_current = pd.read_excel(current_file_path, sheet_name='Sheet1')
df_current["provider"] = current_name
df_current.head()

,name,model,number_input_tokens_min,number_input_tokens_max,number_output_tokens_min,number_output_tokens_max,num_concurrent_requests,server_ttft_s_min,server_ttft_s_p50,server_ttft_s_max,server_end_to_end_latency_s_min,server_end_to_end_latency_s_p50,server_end_to_end_latency_s_max,server_output_token_per_s_min,server_output_token_per_s_p50,server_output_token_per_s_max,server_output_token_per_s_mean,acceptance_rate_min,acceptance_rate_p50,acceptance_rate_max,server_number_input_tokens_p50,server_number_output_tokens_p50,client_ttft_s_min,client_ttft_s_p50,client_ttft_s_max,client_end_to_end_latency_s_min,client_end_to_end_latency_s_p50,client_end_to_end_latency_s_max,client_output_token_per_s_min,client_output_token_per_s_p50,client_output_token_per_s_max,client_output_token_per_s_mean,client_mean_output_token_per_s,num_requests_started,num_completed_requests,num_completed_requests_per_min,number_errors,error_code_frequency,requests_grouping,requests_batching,total_input_tokens,total_output_tokens,representative_batch_size,server_rated_throughput,total_latency,client_output_throughput,client_total_throughput,provider
0,synthetic_0_Meta-Llama-3-3-70B-Instruct_0_1000...,Meta-Llama-3-3-70B-Instruct,39,39,496,496,1,0.0691,0.0723,0.0766,1.5130,1.5266,1.5581,333.5770,337.8375,342.3656,338.0911,NaN,NaN,NaN,39,496,0.1686,0.2808,0.7583,1.6157,1.7567,2.4404,268.7738,338.5953,347.8019,331.8455,274.2699,64,64,33.1778,0,{},"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2496,31744,1,338.0911,115.740043,274.269813,295.835383,Llama 3.3 70B Standalone
1,synthetic_0_Meta-Llama-3-3-70B-Instruct_1_1000...,Meta-Llama-3-3-70B-Instruct,80,80,1000,1000,1,0.0694,0.0719,0.0747,3.0522,3.0900,3.1353,326.1153,328.9144,333.7129,329.0694,NaN,NaN,NaN,80,1000,0.1713,0.3272,0.9903,3.1948,3.3553,4.1669,306.4995,329.5512,334.8144,328.7933,293.6662,64,64,17.6200,0,{},"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",5120,64000,1,329.0694,217.934166,293.666667,317.160000,Llama 3.3 70B Standalone
2,synthetic_0_Meta-Llama-3-3-70B-Instruct_2_1000...,Meta-Llama-3-3-70B-Instruct,1688,1688,1000,1000,1,0.2005,0.2037,0.2061,3.5109,3.5574,3.5797,295.5842,297.5359,300.8714,297.9287,NaN,NaN,NaN,1688,1000,0.3200,0.4593,1.0482,3.6522,3.7930,4.4123,257.5206,298.8217,310.7804,298.5541,259.4201,64,64,15.5652,0,{},"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",108032,64000,1,297.9287,246.704186,259.420000,697.320960,Llama 3.3 70B Standalone
3,synthetic_0_Meta-Llama-3-3-70B-Instruct_3_1000...,Meta-Llama-3-3-70B-Instruct,1955,1955,809,809,1,0.2022,0.2040,0.2059,2.8917,2.9342,2.9575,293.5560,295.3114,299.0403,295.3631,NaN,NaN,NaN,1955,809,0.3300,0.4560,2.3432,3.0551,3.1778,5.0887,248.1662,296.4497,309.5396,293.9722,245.1368,64,64,18.1807,0,{},"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",125120,51776,1,295.3631,211.212990,245.136438,837.524247,Llama 3.3 70B Standalone
4,synthetic_0_Meta-Llama-3-3-70B-Instruct_4_1000...,Meta-Llama-3-3-70B-Instruct,3417,3417,402,402,1,0.3753,0.3767,0.3801,1.4128,1.4308,1.4469,369.8218,379.1978,384.9822,379.1761,NaN,NaN,NaN,3417,402,0.5070,0.6971,2.1206,1.5601,1.7710,3.4071,220.9643,379.2784,443.5685,363.6444,216.2365,64,64,32.2741,0,{},"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",218688,25728,1,379.1761,118.980855,216.236470,2054.246465,Llama 3.3 70B Standalone


#### Read results from alternative source

In [ ]:
"salesforce_xlam2_70b_short_studio_mp1_20250602_1"

alt_type = "run" # "run" or "master"

if alt_type == "run":
    alt_run_name = "salesforce_xlam2_70b_short_studio_20250523_1"
    current_file_path = os.path.join(config["consolidated_results_dir"], f'{alt_run_name}.xlsx')
    df_alt = pd.read_excel(current_file_path, sheet_name='Sheet1')
    df_alt = df_alt[df_alt["model"].isin([t.replace('.', '-') for t in target_models])]
    df_alt["source"] = alt_name
    df_alt.head()
if alt_type == "master":
    df_alt = pd.read_excel('~/Downloads/Master_Benchmarking_Sheet.xlsx', sheet_name='Sheet1',  header=0, index_col=[0, 1, 2, 3]).reset_index()
    df_alt = df_alt[df_alt["model"].isin([t.replace('.', '-') for t in target_models])]
    df_alt["source"] = alt_name
    df_alt.head()

#### Combine dataframes

In [ ]:
df_current.shape, df_alt.shape

((9, 39), (30, 38))

In [ ]:
common_columns = list(set(df_current.columns) & set(df_alt.columns)) # get intersection
common_columns = [c for c in df_current.columns if c in common_columns] # reorder
len(common_columns)

37

In [ ]:
first_columns = ['provider','model','num_input_tokens','num_output_tokens','num_concurrent_requests']
following_columns = [column for column in common_columns if column not in first_columns]
results = pd.concat([
        df_current,
        df_alt
    ], axis=0
)[first_columns + following_columns]
results.head()

### Plot benchmarking charts among providers

You may change the pallette color based on the color that better identifies each provider. You can take the [following link](https://seaborn.pydata.org/tutorial/color_palettes.html) as reference. Also, you will need to update the suptitle to reflect the model you're showing and any other detail. 

In [ ]:
%matplotlib inline

# Pallette for the providers, change colors and provider names as needed
palette = {current_name: 'orange', alt_name: 'tab:purple'}

fields = {
    "Server TTFT (s)": "server_ttft_s_p50",
    "Server E2E Latency (s)": "server_end_to_end_latency_s_p50",
    "Server Tokens/s": "server_output_token_per_s_p50",
}

# Get unique concurrent values
concurrent_values = results['num_concurrent_requests'].unique()
concurrent_values.sort()

plt.clf()
for target_model in target_models:
    # Set up the figure and axes
    fig, axes = plt.subplots(len(concurrent_values), 3, figsize=(20, 6 * len(concurrent_values)), sharex=False)
    fig.suptitle(target_model, fontsize=20)
    for i, concurrent in enumerate(concurrent_values):
        subset = results[results['num_concurrent_requests'] == concurrent]
        subset = subset[subset['model'] == target_model.replace('.', '-')]        
        for j, field in enumerate(fields.keys()):
            ax = axes[i, j]                
            sns.barplot(data=subset, x='num_input_tokens', y=fields[field], hue='provider', ax=ax, palette=palette, errorbar=None)
            ax.set_title(f'{field} for Concurrent Requests: {concurrent}')
            ax.set_xlabel('Input Tokens')
            ax.set_ylabel(field)    
            ax.legend(loc=2)
    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

